In [1]:
import rpwf
from rpwf import database, rpwf
from pathlib import Path
from typing import Dict
from dataclasses import dataclass
import pandas
from sklearn.model_selection import (
    RepeatedStratifiedKFold,
    GridSearchCV,
    cross_val_score
)

In [16]:
tmp_dir = Path("/tmp/Rtmpo4WoiC/vignette3321168c28df")
tmp_dir_posix = str(tmp_dir)
print(tmp_dir_posix)

/tmp/Rtmpo4WoiC/vignette3321168c28df/db.SQLite


In [3]:
os.listdir(tmp_dir)

['board.yml', 'db.SQLite']

In [4]:
@dataclass
class TestArgs:
    db_path: str
    board: str
    cores: int
    inner_n_cv: int
    inner_n_repeats: int
    outer_n_cv: int
    outer_n_repeats: int
    joblib_model: bool

In [5]:
args = TestArgs(
    tmp_dir.joinpath('db.SQLite').as_posix(), 
    tmp_dir.joinpath('board.yml').as_posix(), 
    3, 2, 1, 2, 1, True)
db_obj = database.Base(args.db_path)
board_obj = database.Board(args.board)

Connecting to sqlite:////tmp/Rtmpo4WoiC/vignette3321168c28df/db.SQLite


In [6]:
db_obj.all_wflow()

,wflow_id,model_tag,recipe_tag,result_pin_name,model_pin_name
0,1,xgboost,xgb_base,None,None
1,2,xgboost,xgb_pca,None,None
2,3,svm_rbf,scaled_base,None,None
3,4,glmnet,scaled_base,None,None
4,5,svm_rbf,scaled_pca,wf_5_cv_8754083262181.csv,wf_5_cv_8754083262181.joblib
5,6,glmnet,scaled_pca,wf_6_cv_8731127476396.csv,wf_6_cv_8731127476396.joblib


In [7]:
import sys
from numpy import ravel

wflow_id = 5
wflow_obj = rpwf.Wflow(db_obj, board_obj, wflow_id)
n_cores = args.cores

# Generate the parameters
p_grid = rpwf.RGrid(db_obj, board_obj, wflow_obj).get_grid()

df_obj = rpwf.TrainDf(db_obj, board_obj, wflow_obj)
X, y = df_obj.get_df_X(True), df_obj.get_df_y(True)

if y is None:
    print("No target provided, exiting...")
    sys.exit()

y = ravel(y)

model_type_obj = rpwf.Model(db_obj, board_obj, wflow_obj)
base_learner = rpwf.BaseLearner(wflow_obj, model_type_obj).base_learner
score = wflow_obj._get_par("costs")

Running <class 'sklearn.svm._classes.SVC'>


## nested_resampling.py

In [ ]:
# Nested resampling
inner_cv = RepeatedStratifiedKFold(
    n_splits=args.inner_n_cv, 
    n_repeats=args.inner_n_repeats,
    random_state=wflow_obj.random_state
)
outer_cv = RepeatedStratifiedKFold(
    n_splits=args.outer_n_cv,
    n_repeats=args.outer_n_repeats,
    random_state=wflow_obj.random_state,
)

if p_grid is None:
    print("No tune grid specified, running with default params")
    nested_score = cross_val_score(
        base_learner, X=X, y=y, cv=outer_cv, n_jobs=n_cores, scoring=score
    )

else:
    print("Performing nested-cv using provided Rgrid")
    param_tuner = GridSearchCV(
        estimator=base_learner,
        param_grid=p_grid,
        cv=inner_cv,
        n_jobs=n_cores,
        scoring=score,
    )
    nested_score = cross_val_score(param_tuner, X=X, y=y, cv=outer_cv)

In [ ]:
nested_score

In [ ]:
# if args.export:
    # Export the results
exporter = rpwf.Export(db_obj, board_obj, "nested_cv", wflow_obj)
nested_score_df = pandas.DataFrame(nested_score, columns=[score])

In [ ]:
exporter.export_cv(nested_score_df)
exporter.export_db()

In [ ]:
board_obj.board.pin_read('wf_5_nested_cv_8767807281529.csv')

In [ ]:
db_obj.all_wflow()

## cross_validation.py

In [8]:
cv = RepeatedStratifiedKFold(
    n_splits=5,
    n_repeats=1,
    random_state=wflow_obj.random_state,
)

if p_grid is None:
    print("No tune grid specified, running with default params")
    cv_results = cross_val_score(
        base_learner, X=X, y=y, cv=cv, n_jobs=n_cores, scoring=score
    )
else:
    print("Performing nested-cv using provided Rgrid")
    param_tuner = GridSearchCV(
        estimator=base_learner,
        param_grid=p_grid,
        cv=cv,
        n_jobs=n_cores,
        scoring=score
    )
    param_tuner.fit(X=X, y=y)
    tuning_results = pandas.DataFrame(param_tuner.cv_results_)
    cv_results = tuning_results.loc[tuning_results['rank_test_score'] == 1]

Performing nested-cv using provided Rgrid


In [9]:
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,0.005377,0.000274,0.002581,0.000707,3.174802,0.008621,"{'C': 3.174802103936398, 'gamma': 0.0086210288...",0.945455,0.945455,0.913043,0.960145,0.93599,0.940018,0.015544,1


In [10]:
# if args.export:
    # Export the results
exporter = rpwf.Export(db_obj, board_obj, "cv", wflow_obj)
exporter.export_cv(pandas.DataFrame(cv_results))
if args.joblib_model and param_tuner:
    exporter.export_model(param_tuner.best_estimator_)
exporter.export_db()

Found in db, overwriting
Exporting to db


Writing pin:
Name: 'wf_5_cv_8772869485692.csv'
Version: 20221115T001158Z-43887
Writing pin:
Name: 'wf_5_cv_8772869485692.joblib'
Version: 20221115T001158Z-ccb27


In [13]:
board_obj.board.pin_read('wf_5_cv_8772869485692.csv')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005377,0.000274,0.002581,0.000707,3.174802,0.008621,"{'C': 3.174802103936398, 'gamma': 0.0086210288...",0.945455,0.945455,0.913043,0.960145,0.93599,0.940018,0.015544,1


In [ ]:
db_obj.all_wflow()

## testing the scripts

In [18]:
test_db_path = tmp_dir.joinpath('db.SQLite').as_posix()
test_board_path = tmp_dir.joinpath('board.yml').as_posix()

### nested_resampling

In [20]:
%run -m rpwf.script.nested_resampling $test_db_path -b $test_board_path -h

usage: nested_resampling.py [-h] -b board
                            (-s | -a | -w wflow-id [wflow-id ...]) [-f]
                            [-c cores] [-icv inner-n-cv]
                            [-icr inner-n-repeats] [-ocv outer-n-cv]
                            [-ocr outer-n-repeats]
                            db_path

run nested_cv of the provided wflow_ids

positional arguments:
  db_path               path to the database

optional arguments:
  -h, --help            show this help message and exit
  -b board, --board board
                        path to the yaml file of the board
  -s, --show-wflow      show list of current workflows
  -a, --all-id          run all wflows in the db
  -w wflow-id [wflow-id ...], --wflow-id wflow-id [wflow-id ...]
                        input list of wflows to run
  -f, --force           force runnning of wflows
  -c cores, --cores cores
                        number of cores for parallization
  -icv inner-n-cv, --inner-n-cv inner-n-cv
     

In [ ]:
%run -m rpwf.script.nested_resampling $test_db_path -b $test_board_path -s

In [24]:
%run -m rpwf.script.nested_resampling $test_db_path -b $test_board_path -w 5 -f -c 4 -icv 2 -icr 1 -ocv 2 -ocr 1

running [5]
running wflow 5
Running <class 'sklearn.svm._classes.SVC'>
Performing nested-cv using provided Rgrid
Found in db, overwriting
Exporting to db


Writing pin:
Name: 'wf_5_nested_cv_8772789650357.csv'
Version: 20221115T002302Z-fdd2c


In [ ]:
%run -m rpwf.script.nested_resampling $test_db_path -b $test_board_path -af -c 4 -icv 5 -icr 1 -ocv 5 -ocr 1

### cross_validation

In [25]:
%run -m rpwf.script.cross_validation $test_db_path -b $test_board_path -h

usage: cross_validation.py [-h] -b board
                           (-s | -a | -w wflow-id [wflow-id ...]) [-f]
                           [-c cores] [-ns n-splits] [-nr n-repeats] [-j]
                           db_path

run nested_cv of the provided wflow_ids

positional arguments:
  db_path               path to the database

optional arguments:
  -h, --help            show this help message and exit
  -b board, --board board
                        path to the yaml file of the board
  -s, --show-wflow      show list of current workflows
  -a, --all-id          run all wflows in the db
  -w wflow-id [wflow-id ...], --wflow-id wflow-id [wflow-id ...]
                        input list of wflows to run
  -f, --force           force runnning of wflows
  -c cores, --cores cores
                        number of cores for parallization
  -ns n-splits, --n-splits n-splits
                        number of splits for the cross-validation
  -nr n-repeats, --n-repeats n-repeats
             

In [29]:
%run -m rpwf.script.cross_validation $test_db_path -b $test_board_path -s

   wflow_id model_tag   recipe_tag                   result_pin_name  \
0         1   xgboost     xgb_base  wf_1_nested_cv_8772792900226.csv   
1         2   xgboost      xgb_pca                              None   
2         3   svm_rbf  scaled_base                              None   
3         4    glmnet  scaled_base                              None   
4         5   svm_rbf   scaled_pca         wf_5_cv_8772869485692.csv   
5         5   svm_rbf   scaled_pca  wf_5_nested_cv_8772789660450.csv   
6         6    glmnet   scaled_pca         wf_6_cv_8731127476396.csv   

                 model_pin_name  
0                          None  
1                          None  
2                          None  
3                          None  
4  wf_5_cv_8772869485692.joblib  
5                          None  
6  wf_6_cv_8731127476396.joblib  


In [32]:
%run -m rpwf.script.cross_validation $test_db_path -b $test_board_path -w 5 -f -c 4 -ns 5 -nr 1

running [5]
running wflow 5
Running <class 'sklearn.svm._classes.SVC'>
Performing nested-cv using provided Rgrid
Found in db, overwriting
Exporting to db


Writing pin:
Name: 'wf_5_cv_8772792950116.csv'
Version: 20221115T002639Z-7aee5
